In [560]:
import ultralytics
from ultralytics import YOLO

model = YOLO("yolov8m.pt")

# Creating the data frames 
### (run everything here before anything that works with the data frames)

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import Image, display
import os
from pathlib import Path

#expands table for printing/debug purposes
pd.set_option('display.width', 160)

file = open("NASA_Datasets - Sheet1.csv")

#discards first few lines of no data
for i in range(4):
    file.readline()

chicagoData = []
indianapolisData = []

#read every line of file
while not file.closed:
    dataRow = file.readline()
    if(dataRow == ''):
        file.close()
    else:
        dataRowArray = dataRow.split(',')
        dataRowArray = [string.strip() for string in dataRowArray]
        chicagoData.append(dataRowArray[0 : 6])
        indianapolisData.append(dataRowArray[7 : 13])

#cleaning up indianapolis data
while(indianapolisData[-1][0] == ''):
    #cleaning up empty rows from end caused by larger chicago data
    indianapolisData.pop(-1)

for row in indianapolisData:
    row[5] = row[5].replace('\n', '')

    
#paths to folders of image data
satelliteImageFilePathHeader = "satellite_images/"
streetviewImageFilePathHeader = "streetview_images/"

#set up data frames
chicagoDataFrame = pd.DataFrame(chicagoData)
chicagoDataFrame = chicagoDataFrame.rename(columns={0 : "ID", 1 : "Address", 2 : "SatelliteImageName", 3 : "StreetviewImageName", 4 : "Status", 5 : "Pool"})

indianapolisDataFrame = pd.DataFrame(indianapolisData)
indianapolisDataFrame = indianapolisDataFrame.rename(columns={0 : "ID", 1 : "Address", 2 : "SatelliteImageName", 3 : "StreetviewImageName", 4 : "Status", 5 : "Pool"})

#displaying all the stuff
print(chicagoDataFrame)
print(indianapolisDataFrame)
        


      ID                    Address SatelliteImageName StreetviewImageName     Status   Pool
0      1     13143 S Carondolet Ave              s1001               v1001  Abandoned  FALSE
1      2          7222 S Euclid Ave              s1002               v1002  Abandoned  FALSE
2      3          7236 S Euclid Ave              s1003               v1003      Owned  FALSE
3      4     1344 S Springfield Ave              s1004               v1004  Abandoned  FALSE
4      5             3853 W 14th St              s1005               v1005      Owned  FALSE
..   ...                        ...                ...                 ...        ...    ...
495  496  6531 S Eberhart Ave 60637             s10496              v10496      Owned  FALSE
496  497     4342 W Wilcox St 60624             s10497              v10497      Owned  FALSE
497  498     5331 S Honore St 60609             s10498              v10498  Abandoned  FALSE
498  499  3641 W Douglas Blvd 60623             s10499              v1

In [566]:
#fixing street view image formatting
def fixStreetImage(dataFrame):
    for i in range(dataFrame.shape[0]):
        streetviewImageName = dataFrame.at[i, 'StreetviewImageName']
        if(len(streetviewImageName) < 6):
            streetviewImageName = streetviewImageName.replace('v10','v100')
            streetviewImageName = streetviewImageName.replace('v20','v200')
            dataFrame.at[i, 'StreetviewImageName'] = streetviewImageName

    return dataFrame

chicagoDataFrame = fixStreetImage(chicagoDataFrame)
indianapolisDataFrame = fixStreetImage(indianapolisDataFrame)

#displaying all the stuff
print(chicagoDataFrame)
print(indianapolisDataFrame)


      ID                    Address SatelliteImageName StreetviewImageName     Status   Pool
0      1     13143 S Carondolet Ave              s1001              v10001  Abandoned  FALSE
1      2          7222 S Euclid Ave              s1002              v10002  Abandoned  FALSE
2      3          7236 S Euclid Ave              s1003              v10003      Owned  FALSE
3      4     1344 S Springfield Ave              s1004              v10004  Abandoned  FALSE
4      5             3853 W 14th St              s1005              v10005      Owned  FALSE
..   ...                        ...                ...                 ...        ...    ...
495  496  6531 S Eberhart Ave 60637             s10496              v10496      Owned  FALSE
496  497     4342 W Wilcox St 60624             s10497              v10497      Owned  FALSE
497  498     5331 S Honore St 60609             s10498              v10498  Abandoned  FALSE
498  499  3641 W Douglas Blvd 60623             s10499              v1

In [567]:
#concatenates the data
combinedDataFrame = pd.concat([chicagoDataFrame, indianapolisDataFrame])
print(combinedDataFrame)

      ID                     Address SatelliteImageName StreetviewImageName     Status   Pool
0      1      13143 S Carondolet Ave              s1001              v10001  Abandoned  FALSE
1      2           7222 S Euclid Ave              s1002              v10002  Abandoned  FALSE
2      3           7236 S Euclid Ave              s1003              v10003      Owned  FALSE
3      4      1344 S Springfield Ave              s1004              v10004  Abandoned  FALSE
4      5              3853 W 14th St              s1005              v10005      Owned  FALSE
..   ...                         ...                ...                 ...        ...    ...
286  287        4224 S Post Rd 46239             s20287              v20287      Owned  FALSE
287  288  4366 N ARLINGTON AVE 46226             s20288              v20288  Abandoned  FALSE
288  289        1010 W 27TH ST 46208             s20289              v20289  Abandoned  FALSE
289  290         933 W 27th St 46208             s20290     

# Creating classification data sets
### (creates the folders of training/testing/validation data)

In [ ]:

import random, os.path
import shutil
from pathlib import Path

def createNewTrainingFileDirectory(dataFolder, datagroupFolder, classifyFolder):
    """
    helper function for creating the data set directories

    Parameters
    ----------
    dataFolder : what data set this should belong to
    datagroupFolder : train/val/test
    classifyFolder : what classification this belongs to

    Returns
    -------
    string : path to the directory of the file
        
    """
    path = "classifications/"
    path += dataFolder + "/" + datagroupFolder + "/" + classifyFolder
    
    return path

def createNewTrainingFilePath(name, dataFolder, datagroupFolder, classifyFolder, customFooter = ""):
    """
    helper function for creating the data set paths

    Parameters
    ----------
    name : name of file
    dataFolder : what data set this should belong to
    datagroupFolder : train/val/test
    classifyFolder : what classification this belongs to

    Returns
    -------
    string : path the copied file should have
        
    """
    path = createNewTrainingFileDirectory(dataFolder, datagroupFolder, classifyFolder) + "/" + name + "_" + classifyFolder + customFooter + ".png"
    
    return path
    
#helper function that generates what set of data something should be in
def generateRandomDataGroup(val_odds = 0.1, test_odds = 0.1):
    dataGroupInt = random.uniform(0,1)
    dataGroup = ""
    if(dataGroupInt < val_odds):
        dataGroup = "val"
    elif(dataGroupInt < val_odds + test_odds):
        dataGroup = "test"
    else:
        dataGroup = "train"
    return dataGroup


#paths to folders of image data
satelliteImageFilePathHeader = "satellite_images/"
streetviewImageFilePathHeader = "streetview_images/"

print("helper functions created")

helper functions created


In [ ]:
#abandonned vs owned streetview dataset creation


random.seed(42) #consistent results when generating training data

#turning the streetview image data into classification training data
for index,row in combinedDataFrame.iterrows():
    streetviewImageName = row['StreetviewImageName']
    path = streetviewImageFilePathHeader + streetviewImageName + ".png"

    abandoned = row['Status'].strip().lower()
    
    datagroupFolder = generateRandomDataGroup()
    
    newPath = createNewTrainingFilePath(streetviewImageName, "streetview_abandonedness", datagroupFolder, abandoned)
    newDirectory = createNewTrainingFileDirectory("streetview_abandonedness", datagroupFolder, abandoned)

    if(os.path.isfile(path)):
        Path(newDirectory).mkdir(parents=True, exist_ok=True)
        shutil.copyfile(path, newPath)


#same as above but the mirrored street view images
for index,row in combinedDataFrame.iterrows():
    streetviewImageName = row['StreetviewImageName']
    path = "streetview_images_mirrored/" + streetviewImageName + ".png"

    abandoned = row['Status'].strip().lower()
    
    datagroupFolder = generateRandomDataGroup()
    
    newPath = createNewTrainingFilePath(streetviewImageName, "streetview_abandonedness", datagroupFolder, abandoned, "_m")
    newDirectory = createNewTrainingFileDirectory("streetview_abandonedness", datagroupFolder, abandoned)

    if(os.path.isfile(path)):
        Path(newDirectory).mkdir(parents=True, exist_ok=True)
        shutil.copyfile(path, newPath)

In [327]:
#abandonned vs owned streetview dataset creation (clean, no mirrored images


random.seed(42) #consistent results when generating training data

#turning the streetview image data into classification training data
for index,row in combinedDataFrame.iterrows():
    streetviewImageName = row['StreetviewImageName']
    path = streetviewImageFilePathHeader + streetviewImageName + ".png"

    abandoned = row['Status'].strip().lower()
    
    datagroupFolder = generateRandomDataGroup()
    
    newPath = createNewTrainingFilePath(streetviewImageName, "streetview_abandonedness_clean", datagroupFolder, abandoned)
    newDirectory = createNewTrainingFileDirectory("streetview_abandonedness_clean", datagroupFolder, abandoned)

    if(os.path.isfile(path)):
        Path(newDirectory).mkdir(parents=True, exist_ok=True)
        shutil.copyfile(path, newPath)



In [ ]:
#abandonned vs owned satelliteview dataset creation

random.seed(43) #consistent results when generating training data

#turning the streetview image data into classification training data
for index,row in combinedDataFrame.iterrows():
    imageName = row['SatelliteImageName']
    path = satelliteImageFilePathHeader + imageName + ".png"

    abandoned = row['Status'].strip().lower()
    
    datagroupFolder = generateRandomDataGroup()
    
    newPath = createNewTrainingFilePath(imageName, "satellite_abandonedness", datagroupFolder, abandoned)
    newDirectory = createNewTrainingFileDirectory("satellite_abandonedness", datagroupFolder, abandoned)

    if(os.path.isfile(path)):
        Path(newDirectory).mkdir(parents=True, exist_ok=True)
        shutil.copyfile(path, newPath)


#same as above but for mirrored images
for index,row in combinedDataFrame.iterrows():
    imageName = row['SatelliteImageName']
    path = "satellite_images_mirrored/" + imageName + ".png"

    abandoned = row['Status'].strip().lower()
    
    datagroupFolder = generateRandomDataGroup()
    
    newPath = createNewTrainingFilePath(imageName, "satellite_abandonedness", datagroupFolder, abandoned, "_m")
    newDirectory = createNewTrainingFileDirectory("satellite_abandonedness", datagroupFolder, abandoned)

    if(os.path.isfile(path)):
        Path(newDirectory).mkdir(parents=True, exist_ok=True)
        shutil.copyfile(path, newPath)


In [ ]:
#abandonned vs owned stitched dataset creation

random.seed(44) #consistent results when generating training data

#turning the streetview image data into classification training data
for index,row in combinedDataFrame.iterrows():
    streetviewImageName = row['StreetviewImageName']
    satelliteImageName = row['SatelliteImageName']
    imageName = f"{streetviewImageName}_{satelliteImageName}"
    path = f"streetsatellitestiched/{imageName}.png"

    abandoned = row['Status'].strip().lower()
    
    datagroupFolder = generateRandomDataGroup()
    
    newPath = createNewTrainingFilePath(imageName, "stitched_abandonedness", datagroupFolder, abandoned)
    newDirectory = createNewTrainingFileDirectory("stitched_abandonedness", datagroupFolder, abandoned)

    if(os.path.isfile(path)):
        Path(newDirectory).mkdir(parents=True, exist_ok=True)
        shutil.copyfile(path, newPath)


#above but mirrored
for index,row in combinedDataFrame.iterrows():
    streetviewImageName = row['StreetviewImageName']
    satelliteImageName = row['SatelliteImageName']
    imageName = f"{streetviewImageName}_{satelliteImageName}"
    path = f"streetsatellitestiched_mirrored/{imageName}.png"

    abandoned = row['Status'].strip().lower()
    
    datagroupFolder = generateRandomDataGroup()
    
    newPath = createNewTrainingFilePath(imageName, "stitched_abandonedness", datagroupFolder, abandoned, "_m")
    newDirectory = createNewTrainingFileDirectory("stitched_abandonedness", datagroupFolder, abandoned)

    if(os.path.isfile(path)):
        Path(newDirectory).mkdir(parents=True, exist_ok=True)
        shutil.copyfile(path, newPath)



In [ ]:
#abandonned vs owned stitched dataset creation (Clean, meaning no mirrored or flipped images)

random.seed(44) #consistent results when generating training data

#turning the streetview image data into classification training data
for index,row in combinedDataFrame.iterrows():
    streetviewImageName = row['StreetviewImageName']
    satelliteImageName = row['SatelliteImageName']
    imageName = f"{streetviewImageName}_{satelliteImageName}"
    path = f"streetsatellitestiched/{imageName}.png"

    abandoned = row['Status'].strip().lower()
    
    datagroupFolder = generateRandomDataGroup()
    
    newPath = createNewTrainingFilePath(imageName, "stitched_abandonedness_clean", datagroupFolder, abandoned)
    newDirectory = createNewTrainingFileDirectory("stitched_abandonedness_clean", datagroupFolder, abandoned)

    if(os.path.isfile(path)):
        Path(newDirectory).mkdir(parents=True, exist_ok=True)
        shutil.copyfile(path, newPath)


In [ ]:
#abandonned vs owned overlay dataset creation (Clean, meaning no mirrored or flipped images)

random.seed(45) #consistent results when generating training data

#turning the streetview image data into classification training data
for index,row in combinedDataFrame.iterrows():
    streetviewImageName = row['StreetviewImageName']
    satelliteImageName = row['SatelliteImageName']
    imageName = f"{streetviewImageName}_{satelliteImageName}"
    path = f"streetsatelliteoverlayed/{imageName}.png"

    abandoned = row['Status'].strip().lower()
    
    datagroupFolder = generateRandomDataGroup()
    
    newPath = createNewTrainingFilePath(imageName, "overlay_abandonedness", datagroupFolder, abandoned)
    newDirectory = createNewTrainingFileDirectory("overlay_abandonedness", datagroupFolder, abandoned)

    if(os.path.isfile(path)):
        Path(newDirectory).mkdir(parents=True, exist_ok=True)
        shutil.copyfile(path, newPath)


In [ ]:
#chicago vs indianapolis streetview dataset creation
import random, os.path
import shutil

random.seed(13)#consistent results when generating training data

#turning the streetview image data into classification training data
for i in range(chicagoDataFrame.shape[0]):
    streetviewImageName = chicagoDataFrame.at[i, 'StreetviewImageName']
    path = streetviewImageFilePathHeader + streetviewImageName + ".png"

    dataGroup = generateRandomDataGroup()
    
    newPath = createNewTrainingFilePath(streetviewImageName, "chi_vs_ind", dataGroup, 'chicago')
    
    shutil.copyfile(path, newPath)

for i in range(indianapolisDataFrame.shape[0]):
    streetviewImageName = indianapolisDataFrame.at[i, 'StreetviewImageName']
    path = streetviewImageFilePathHeader + streetviewImageName + ".png"

    dataGroup = generateRandomDataGroup()
    
    newPath = createNewTrainingFilePath(streetviewImageName, "chi_vs_ind", dataGroup, 'indianapolis')

    if(os.path.isfile(path)):
        shutil.copyfile(path, newPath)


In [ ]:
#street view vs satellite view dataset creation
import random
from pathlib import Path
import shutil


#turning the streetview and satellite image data into classification training data
for i in range(chicagoDataFrame.shape[0]):
    satelliteviewImageName = chicagoDataFrame.at[i, 'SatelliteImageName']
    path = satelliteImageFilePathHeader + satelliteviewImageName + ".png"
    
    validating = random.randint(1, 8) == 1
    newPath = createNewTrainingFilePath(satelliteviewImageName, "satellite_vs_street", True, validating)
    
    shutil.copyfile(path, newPath)
    
    streetviewImageName = chicagoDataFrame.at[i, 'StreetviewImageName']
    path = streetviewImageFilePathHeader + streetviewImageName + ".png"
    
    validating = random.randint(1, 8) == 1
    newPath = createNewTrainingFilePath(streetviewImageName, "satellite_vs_street", False, validating)
    
    shutil.copyfile(path, newPath)

In [232]:
#pool vs no pool dataset creation
import random
from pathlib import Path
import shutil

def createPoolDataFromFolder(folderPath, footer = ''):
    for index,row in combinedDataFrame.iterrows():
        satelliteviewImageName = row['SatelliteImageName']
        path = folderPath + satelliteviewImageName + ".png"
    
        hasPool = row['Pool'] == 'TRUE'
        if(hasPool):
            poolStatus = "pool"
            dataGroup = generateRandomDataGroup(val_odds = .15, test_odds = .1)
        else:
            poolStatus = "no_pool"
            #intentionally put most of the no_pool data stuff into testing so the training and validation data is more balanced
            dataGroup = generateRandomDataGroup(val_odds = .02, test_odds = .90)
            
        newPath = createNewTrainingFilePath(satelliteviewImageName, "satellite_poolhaving", dataGroup, poolStatus, footer)
        newDirectory = createNewTrainingFileDirectory("satellite_poolhaving", dataGroup, poolStatus)
        
        if(os.path.isfile(path)):
            Path(newDirectory).mkdir(parents=True, exist_ok=True)
            shutil.copyfile(path, newPath)

random.seed(101) #consistent test data generation

#turning the satellite image data into classification training data
createPoolDataFromFolder("satellite_images/")

#same as above but for mirrored data
createPoolDataFromFolder("satellite_images_mirrored/", '_m')

#same as above and above but for mirrored vertically data
createPoolDataFromFolder("satellite_images_mirrored_vert/", '_mv')

#same as above and above and above but for flipped data
createPoolDataFromFolder("satellite_images_flipped/", '_f')
    

In [366]:
#pool vs no pool dataset creation (Clean, no flipped or mirror images)
import random
from pathlib import Path
import shutil

random.seed(101) #consistent test data generation
#turning the satellite image data into classification training data
for index,row in combinedDataFrame.iterrows():
    satelliteviewImageName = row['SatelliteImageName']
    path = satelliteImageFilePathHeader + satelliteviewImageName + ".png"

    hasPool = row['Pool'] == 'TRUE'
    if(hasPool):
        poolStatus = "pool"
        dataGroup = generateRandomDataGroup(val_odds = .3, test_odds = .15)
    else:
        poolStatus = "no_pool"
        #intentionally put most of the no_pool data stuff into testing so the training and validation data is more balanced
        dataGroup = generateRandomDataGroup(val_odds = .05, test_odds = .9)
        
    newPath = createNewTrainingFilePath(satelliteviewImageName, "satellite_poolhaving_clean", dataGroup, poolStatus)
    newDirectory = createNewTrainingFileDirectory("satellite_poolhaving_clean", dataGroup, poolStatus)
    
    if(os.path.isfile(path)):
        Path(newDirectory).mkdir(parents=True, exist_ok=True)
        shutil.copyfile(path, newPath)
    

# Abadonness Classification Model

## Street View

In [493]:
#training abandonnessModel for street view

#modelPath = "yolov8m-cls.pt"  #uncomment to restart training from scratch
#abandonnessModel = YOLO(modelPath)
abandonnessModel.train(data="classifications/streetview_abandonedness", epochs = 100, fliplr = 0, shear=10)

New https://pypi.org/project/ultralytics/8.3.78 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
engine/trainer: task=classify, mode=train, model=runs/classify/train138/weights/last.pt, data=classifications/streetview_abandonedness, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train139, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes

train: Scanning /home/jupyter/classifications/streetview_abandonedness/train... 1234 images, 0 corrupt: 100%|██████████|
val: Scanning /home/jupyter/classifications/streetview_abandonedness/val... 150 images, 0 corrupt: 100%|██████████| 150/


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 38 weight(decay=0.0), 39 weight(decay=0.0005), 39 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs/classify/train139
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100      2.45G     0.2894          2        224: 100%|██████████| 78/78 [00:03<00:00, 25.00it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 93.24it/s]

                   all      0.753          1


...

                   all      0.907          1



100 epochs completed in 0.083 hours.
Optimizer stripped from runs/classify/train139/weights/last.pt, 31.7MB
Optimizer stripped from runs/classify/train139/weights/best.pt, 31.7MB

Validating runs/classify/train139/weights/best.pt...
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,765,218 parameters, 0 gradients, 41.6 GFLOPs
train: /home/jupyter/classifications/streetview_abandonedness/train... found 1234 images in 2 classes ✅ 
val: /home/jupyter/classifications/streetview_abandonedness/val... found 150 images in 2 classes ✅ 
test: /home/jupyter/classifications/streetview_abandonedness/test... found 136 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 48.42it/s]


                   all      0.913          1
Speed: 0.0ms preprocess, 0.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train139


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d73ac6e2710>
curves: []
curves_results: []
fitness: 0.9566666781902313
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9133333563804626, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9566666781902313}
save_dir: PosixPath('runs/classify/train139')
speed: {'preprocess': 0.04946549733479818, 'inference': 0.6072473526000977, 'loss': 0.00025272369384765625, 'postprocess': 0.00038623809814453125}
task: 'classify'
top1: 0.9133333563804626
top5: 1.0

In [494]:
#getting pretrained model
modelPath = "runs/classify/train139/weights/last.pt"
abandonnessModel = YOLO(modelPath) # street view model

In [495]:
#predicting from test data (street view)
testFolderPath = "classifications/streetview_abandonedness/test/abandoned"
abandonedTestResults = abandonnessModel.predict(testFolderPath)
testFolderPath = "classifications/streetview_abandonedness/test/owned"
ownedTestResults = abandonnessModel.predict(testFolderPath)


image 1/60 /home/jupyter/classifications/streetview_abandonedness/test/abandoned/v10015_abandoned_m.png: 224x224 owned 0.97, abandoned 0.03, 3.8ms
image 2/60 /home/jupyter/classifications/streetview_abandonedness/test/abandoned/v10078_abandoned_m.png: 224x224 abandoned 0.98, owned 0.02, 3.7ms
image 3/60 /home/jupyter/classifications/streetview_abandonedness/test/abandoned/v10082_abandoned_m.png: 224x224 owned 0.78, abandoned 0.22, 3.7ms
image 4/60 /home/jupyter/classifications/streetview_abandonedness/test/abandoned/v10088_abandoned_m.png: 224x224 abandoned 1.00, owned 0.00, 3.8ms
image 5/60 /home/jupyter/classifications/streetview_abandonedness/test/abandoned/v10091_abandoned.png: 224x224 abandoned 0.98, owned 0.02, 3.7ms
image 6/60 /home/jupyter/classifications/streetview_abandonedness/test/abandoned/v10120_abandoned.png: 224x224 abandoned 1.00, owned 0.00, 3.7ms
image 7/60 /home/jupyter/classifications/streetview_abandonedness/test/abandoned/v10143_abandoned_m.png: 224x224 abandone

In [496]:
analyzeAbandonedTestData(abandonedTestResults, ownedTestResults)

Accuracy:  121 / 136  =  0.89

Abandonness precision:  0.869
Owned precision:  0.907
Abandonness recall:  0.883
Owned recall:  0.895

Abandonness f1 score:  0.876
Owned f1 score:  0.901


## Satellite View

In [545]:
#training abandonnessModel for satellite view

#abandonnessSatelliteModel = YOLO("yolov8m-cls.pt") #uncomment to restart training from scratch
abandonnessSatelliteModel.train(data="classifications/satellite_abandonedness", epochs = 100, flipud=0, fliplr=0,translate = 0.3, scale = 0.2)

New https://pypi.org/project/ultralytics/8.3.78 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
engine/trainer: task=classify, mode=train, model=runs/classify/train148/weights/best.pt, data=classifications/satellite_abandonedness, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train149, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=

train: Scanning /home/jupyter/classifications/satellite_abandonedness/train... 1284 images, 0 corrupt: 100%|██████████| 
val: Scanning /home/jupyter/classifications/satellite_abandonedness/val... 159 images, 0 corrupt: 100%|██████████| 159/1


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 38 weight(decay=0.0), 39 weight(decay=0.0005), 39 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs/classify/train149
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100      2.28G     0.6878          4        224: 100%|██████████| 81/81 [00:02<00:00, 27.82it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 98.39it/s]

                   all      0.591          1


...

                   all      0.862          1



100 epochs completed in 0.084 hours.
Optimizer stripped from runs/classify/train149/weights/last.pt, 31.7MB
Optimizer stripped from runs/classify/train149/weights/best.pt, 31.7MB

Validating runs/classify/train149/weights/best.pt...
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,765,218 parameters, 0 gradients, 41.6 GFLOPs
train: /home/jupyter/classifications/satellite_abandonedness/train... found 1284 images in 3 classes: ERROR ❌️ requires 2 classes, not 3
val: /home/jupyter/classifications/satellite_abandonedness/val... found 159 images in 2 classes ✅ 
test: /home/jupyter/classifications/satellite_abandonedness/test... found 141 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 47.66it/s]


                   all      0.874          1
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train149


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d7390b8b6a0>
curves: []
curves_results: []
fitness: 0.9371069073677063
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8742138147354126, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9371069073677063}
save_dir: PosixPath('runs/classify/train149')
speed: {'preprocess': 0.06009647681278253, 'inference': 0.5296821114402148, 'loss': 0.0004123591776913817, 'postprocess': 0.005716047946761989}
task: 'classify'
top1: 0.8742138147354126
top5: 1.0

In [546]:
#getting pretrained model
modelPath = "runs/classify/train149/weights/best.pt"
abandonnessSatelliteModel = YOLO(modelPath) # satellite view

In [547]:
#predicting from test data (Satellite)
testFolderPath = "classifications/satellite_abandonedness/test/abandoned"
abandonedTestResults = abandonnessSatelliteModel.predict(testFolderPath)
testFolderPath = "classifications/satellite_abandonedness/test/owned"
ownedTestResults = abandonnessSatelliteModel.predict(testFolderPath)


image 1/65 /home/jupyter/classifications/satellite_abandonedness/test/abandoned/s1006_abandoned_m.png: 224x224 abandoned 0.72, owned 0.28, 3.8ms
image 2/65 /home/jupyter/classifications/satellite_abandonedness/test/abandoned/s10103_abandoned.png: 224x224 abandoned 0.90, owned 0.10, 3.7ms
image 3/65 /home/jupyter/classifications/satellite_abandonedness/test/abandoned/s10113_abandoned_m.png: 224x224 abandoned 1.00, owned 0.00, 3.8ms
image 4/65 /home/jupyter/classifications/satellite_abandonedness/test/abandoned/s10132_abandoned.png: 224x224 owned 0.51, abandoned 0.49, 3.7ms
image 5/65 /home/jupyter/classifications/satellite_abandonedness/test/abandoned/s10143_abandoned.png: 224x224 abandoned 0.98, owned 0.02, 3.9ms
image 6/65 /home/jupyter/classifications/satellite_abandonedness/test/abandoned/s10151_abandoned.png: 224x224 owned 0.62, abandoned 0.38, 3.7ms
image 7/65 /home/jupyter/classifications/satellite_abandonedness/test/abandoned/s10162_abandoned_m.png: 224x224 abandoned 1.00, owne

In [548]:
analyzeAbandonedTestData(abandonedTestResults, ownedTestResults)

Accuracy:  116 / 141  =  0.823

Abandonness precision:  0.778
Owned precision:  0.87
Abandonness recall:  0.862
Owned recall:  0.789

Abandonness f1 score:  0.818
Owned f1 score:  0.828


## Stitched View

In [549]:
#training abandonnessModel for stitched view

#abandonnessStitchedModel = YOLO("yolov8m-cls.pt")#uncomment to restart training from scratch
trainingResults = abandonnessStitchedModel.train(data="classifications/stitched_abandonedness", epochs = 200, fliplr = 0, mosaic = 0)

New https://pypi.org/project/ultralytics/8.3.78 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=classifications/stitched_abandonedness, epochs=200, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train150, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False,

train: Scanning /home/jupyter/classifications/stitched_abandonedness/train... 1194 images, 0 corrupt: 100%|██████████| 1
val: Scanning /home/jupyter/classifications/stitched_abandonedness/val... 171 images, 0 corrupt: 100%|██████████| 171/17


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 38 weight(decay=0.0), 39 weight(decay=0.0005), 39 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs/classify/train150
Starting training for 200 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/200      2.34G     0.6345         10        224: 100%|██████████| 75/75 [00:03<00:00, 24.94it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 6/6 [00:00<00:00, 93.32it/s]

                   all      0.749          1


...

    200/200      1.99G    0.01869         10        224: 100%|██████████| 75/75 [00:02<00:00, 32.17it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 6/6 [00:00<00:00, 93.74it/s]

                   all      0.854          1



200 epochs completed in 0.164 hours.
Optimizer stripped from runs/classify/train150/weights/last.pt, 31.7MB
Optimizer stripped from runs/classify/train150/weights/best.pt, 31.7MB

Validating runs/classify/train150/weights/best.pt...
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,765,218 parameters, 0 gradients, 41.6 GFLOPs
train: /home/jupyter/classifications/stitched_abandonedness/train... found 1194 images in 2 classes ✅ 
val: /home/jupyter/classifications/stitched_abandonedness/val... found 171 images in 2 classes ✅ 
test: /home/jupyter/classifications/stitched_abandonedness/test... found 155 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 6/6 [00:00<00:00, 53.76it/s]


                   all      0.895          1
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train150


In [556]:
#getting pretrained model
modelPath = "runs/classify/train150/weights/best.pt"
abandonnessStitchedModel = YOLO(modelPath) # stitched view

In [557]:
#predicting from test data (stitched)
testFolderPath = "classifications/stitched_abandonedness/test/abandoned"
abandonedTestResults = abandonnessStitchedModel.predict(testFolderPath)
testFolderPath = "classifications/stitched_abandonedness/test/owned"
ownedTestResults = abandonnessStitchedModel.predict(testFolderPath)


image 1/85 /home/jupyter/classifications/stitched_abandonedness/test/abandoned/v10001_s1001_abandoned_m.png: 224x224 abandoned 1.00, owned 0.00, 4.0ms
image 2/85 /home/jupyter/classifications/stitched_abandonedness/test/abandoned/v10004_s1004_abandoned.png: 224x224 abandoned 1.00, owned 0.00, 3.9ms
image 3/85 /home/jupyter/classifications/stitched_abandonedness/test/abandoned/v10007_s1007_abandoned.png: 224x224 abandoned 0.98, owned 0.02, 3.9ms
image 4/85 /home/jupyter/classifications/stitched_abandonedness/test/abandoned/v10009_s1009_abandoned.png: 224x224 abandoned 1.00, owned 0.00, 3.8ms
image 5/85 /home/jupyter/classifications/stitched_abandonedness/test/abandoned/v10053_s1053_abandoned_m.png: 224x224 abandoned 1.00, owned 0.00, 3.9ms
image 6/85 /home/jupyter/classifications/stitched_abandonedness/test/abandoned/v10060_s1060_abandoned_m.png: 224x224 abandoned 0.96, owned 0.04, 3.8ms
image 7/85 /home/jupyter/classifications/stitched_abandonedness/test/abandoned/v10064_s1064_abandon

In [558]:
analyzeAbandonedTestData(abandonedTestResults, ownedTestResults)

Accuracy:  131 / 155  =  0.845

Abandonness precision:  0.835
Owned precision:  0.859
Abandonness recall:  0.894
Owned recall:  0.786

Abandonness f1 score:  0.864
Owned f1 score:  0.821


## Overlay s&s View 

In [477]:
#training abandonnessModel for overlay view

abandonnessOverlayModel = YOLO("yolov8m-cls.pt")#uncomment to restart training from scratch
abandonnessOverlayModel.train(data="classifications/overlay_abandonedness", epochs = 200)

New https://pypi.org/project/ultralytics/8.3.78 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=classifications/overlay_abandonedness, epochs=200, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train134, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, 

train: Scanning /home/jupyter/classifications/overlay_abandonedness/train... 609 images, 0 corrupt: 100%|██████████| 609
val: Scanning /home/jupyter/classifications/overlay_abandonedness/val... 77 images, 0 corrupt: 100%|██████████| 77/77 [0


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 38 weight(decay=0.0), 39 weight(decay=0.0005), 39 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs/classify/train134
Starting training for 200 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/200      2.06G     0.6538          1        224: 100%|██████████| 39/39 [00:01<00:00, 24.33it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00, 101.02it/s]

                   all      0.727          1


...

                   all      0.701          1



200 epochs completed in 0.096 hours.
Optimizer stripped from runs/classify/train134/weights/last.pt, 31.7MB
Optimizer stripped from runs/classify/train134/weights/best.pt, 31.7MB

Validating runs/classify/train134/weights/best.pt...
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,765,218 parameters, 0 gradients, 41.6 GFLOPs
train: /home/jupyter/classifications/overlay_abandonedness/train... found 609 images in 4 classes: ERROR ❌️ requires 2 classes, not 4
val: /home/jupyter/classifications/overlay_abandonedness/val... found 77 images in 2 classes ✅ 
test: /home/jupyter/classifications/overlay_abandonedness/test... found 76 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00, 98.37it/s]


                   all      0.753          1
Speed: 0.0ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train134


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d737a30b550>
curves: []
curves_results: []
fitness: 0.8766233623027802
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.7532467246055603, 'metrics/accuracy_top5': 1.0, 'fitness': 0.8766233623027802}
save_dir: PosixPath('runs/classify/train134')
speed: {'preprocess': 0.04307945053298752, 'inference': 0.3042840338372565, 'loss': 0.00037775411234273543, 'postprocess': 0.0005047042648513596}
task: 'classify'
top1: 0.7532467246055603
top5: 1.0

## Analysis

In [106]:
#analyzing test data
def analyzeAbandonedTestData(abandonedTestResults, ownedTestResults):
    correctAbandons = 0
    correctOwneds = 0
    falseAbandons = 0
    falseOwneds = 0
    totalTests = 0
    
    for result in abandonedTestResults:
        if(result.names[result.probs.top1] == 'abandoned'):
            correctAbandons += 1
        else:
            falseOwneds += 1
        totalTests += 1
    
    
    for result in ownedTestResults:
        if(result.names[result.probs.top1] == 'owned'):
            correctOwneds += 1
        else:
            falseAbandons += 1
        totalTests += 1
    
    #calculating accuracy (how accurate the model is at getting the correct results)
    correctTests = correctAbandons + correctOwneds
    accuracy = correctTests/totalTests
    
    #calculating precision (how good the model is at only saying a thing's there if it is there)
    abandonnessPrecision = correctAbandons / (correctAbandons + falseAbandons)
    ownedPrecision = correctOwneds / (correctOwneds + falseOwneds)
    
    #calculating recall (how good the model is at recognizing a thing is there)
    abandonnessRecall = correctAbandons / (correctAbandons + falseOwneds)
    ownedRecall = correctOwneds / (correctOwneds + falseAbandons) 
    
    #calculating f1 score 
    abandonnessF1 = 2 * (abandonnessRecall * abandonnessPrecision)/(abandonnessRecall + abandonnessPrecision)
    ownedF1 = 2 * (ownedRecall * ownedPrecision)/(ownedRecall + ownedPrecision)
    
    print("Accuracy: ", correctTests, "/", totalTests, " = ", round(accuracy, 3))
    print()
    print("Abandonness precision: ", round(abandonnessPrecision, 3))
    print("Owned precision: ", round(ownedPrecision, 3))
    print("Abandonness recall: ", round(abandonnessRecall, 3))
    print("Owned recall: ", round(ownedRecall, 3))
    print()
    print("Abandonness f1 score: ", round(abandonnessF1, 3))
    print("Owned f1 score: ", round(ownedF1, 3))

In [306]:
# analysis from multiple models at the same time

for index,row in combinedDataFrame.iterrows():
    streetviewImageName = row['StreetviewImageName']
    satelliteImageName = row['SatelliteImageName']

    streetviewImagePath = f"streetview_images/{streetviewImageName}.png"
    satelliteImagePath = f"satellite_images/{satelliteImageName}.png"

    streetResult = abandonnessModel.predict(streetviewImagePath)[0]
    satResult = abandonnessSatelliteModel.predict(satelliteImagePath)[0]

    if(streetResult.probs.top1 == satResult.probs.top1):
        combinedDecision = streetResult.names[streetResult.probs.top1]
        print(combinedDecision)
    else:
        print("!!!!!!!!!!!!!!!!!!!!!!")
    
    if(index > 20):
        break
    

    


image 1/1 /home/jupyter/streetview_images/v10001.png: 224x224 abandoned 1.00, owned 0.00, 4.7ms
Speed: 6.3ms preprocess, 4.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /home/jupyter/satellite_images/s1001.png: 224x224 abandoned 0.97, owned 0.03, 4.8ms
Speed: 3.2ms preprocess, 4.8ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
abandoned

image 1/1 /home/jupyter/streetview_images/v10002.png: 224x224 abandoned 1.00, owned 0.00, 3.7ms
Speed: 2.0ms preprocess, 3.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /home/jupyter/satellite_images/s1002.png: 224x224 abandoned 1.00, owned 0.00, 3.7ms
Speed: 2.0ms preprocess, 3.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
abandoned

image 1/1 /home/jupyter/streetview_images/v10003.png: 224x224 owned 1.00, abandoned 0.00, 3.7ms
Speed: 3.2ms preprocess, 3.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /home/ju

# Satellite vs Street view classification model

In [519]:

satvsstrModel = YOLO("yolov8m-cls.pt")
trainingResults = satvsstrModel.train(data="classifications/satellite_vs_street", epochs = 5)
print(trainingResults)

New https://pypi.org/project/ultralytics/8.3.78 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=classifications/satellite_vs_street, epochs=5, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train144, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embe

train: Scanning /home/jupyter/classifications/satellite_vs_street/train... 868 images, 0 corrupt: 100%|██████████| 868/8
val: Scanning /home/jupyter/classifications/satellite_vs_street/val... 137 images, 0 corrupt: 100%|██████████| 137/137 [

val: New cache created: /home/jupyter/classifications/satellite_vs_street/val.cache


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 38 weight(decay=0.0), 39 weight(decay=0.0005), 39 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs/classify/train144
Starting training for 5 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 95.84it/s]

...

               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 95.88it/s]

                   all          0          1



5 epochs completed in 0.004 hours.
Optimizer stripped from runs/classify/train144/weights/last.pt, 31.7MB
Optimizer stripped from runs/classify/train144/weights/best.pt, 31.7MB

Validating runs/classify/train144/weights/best.pt...
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,766,499 parameters, 0 gradients, 41.6 GFLOPs
train: /home/jupyter/classifications/satellite_vs_street/train... found 868 images in 4 classes: ERROR ❌️ requires 3 classes, not 4
val: /home/jupyter/classifications/satellite_vs_street/val... found 137 images in 4 classes: ERROR ❌️ requires 3 classes, not 4
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 42.73it/s]


                   all          0          1
Speed: 0.0ms preprocess, 0.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train144
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d7390e212d0>
curves: []
curves_results: []
fitness: 0.5
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.0, 'metrics/accuracy_top5': 1.0, 'fitness': 0.5}
save_dir: PosixPath('runs/classify/train144')
speed: {'preprocess': 0.04110197081182995, 'inference': 0.5891897382527372, 'loss': 0.00043507039982036954, 'postprocess': 0.003497966014555771}
task: 'classify'
top1: 0.0
top5: 1.0


In [520]:

streetViewPaths = [streetviewImageFilePathHeader + path + ".png" for path in indianapolisDataFrame.loc[:100, 'StreetviewImageName']]
satelliteViewPaths = [satelliteImageFilePathHeader + path + ".png" for path in indianapolisDataFrame.loc[:100, 'SatelliteImageName']]

results = satvsstrModel.predict(streetViewPaths + satelliteViewPaths)


0: 224x224 streetview 1.00, satellite 0.00, .ipynb_checkpoints 0.00, 0.3ms
1: 224x224 streetview 1.00, .ipynb_checkpoints 0.00, satellite 0.00, 0.3ms
2: 224x224 streetview 1.00, satellite 0.00, .ipynb_checkpoints 0.00, 0.3ms
3: 224x224 streetview 1.00, .ipynb_checkpoints 0.00, satellite 0.00, 0.3ms
4: 224x224 streetview 1.00, .ipynb_checkpoints 0.00, satellite 0.00, 0.3ms
5: 224x224 streetview 1.00, satellite 0.00, .ipynb_checkpoints 0.00, 0.3ms
6: 224x224 streetview 1.00, .ipynb_checkpoints 0.00, satellite 0.00, 0.3ms
7: 224x224 streetview 1.00, satellite 0.00, .ipynb_checkpoints 0.00, 0.3ms
8: 224x224 streetview 1.00, .ipynb_checkpoints 0.00, satellite 0.00, 0.3ms
9: 224x224 streetview 1.00, .ipynb_checkpoints 0.00, satellite 0.00, 0.3ms
10: 224x224 streetview 1.00, .ipynb_checkpoints 0.00, satellite 0.00, 0.3ms
11: 224x224 streetview 1.00, .ipynb_checkpoints 0.00, satellite 0.00, 0.3ms
12: 224x224 streetview 1.00, .ipynb_checkpoints 0.00, satellite 0.00, 0.3ms
13: 224x224 streetvie

In [522]:
print(trainingResults)
print(trainingResults.save_dir)

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d7390e212d0>
curves: []
curves_results: []
fitness: 0.5
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.0, 'metrics/accuracy_top5': 1.0, 'fitness': 0.5}
save_dir: PosixPath('runs/classify/train144')
speed: {'preprocess': 0.04110197081182995, 'inference': 0.5891897382527372, 'loss': 0.00043507039982036954, 'postprocess': 0.003497966014555771}
task: 'classify'
top1: 0.0
top5: 1.0
runs/classify/train144


# Pool vs no pool classification model
(not enough "pool" data for this one probably)

In [384]:
#modelPath = "runs/classify/train21/weights/best.pt"

#poolModel = YOLO("yolov8m-cls.pt") #uncomment to restart training
poolModel.train(data="classifications/satellite_poolhaving", epochs = 100, degrees = 90, flipud = 0.0, fliplr = 0.0, translate = 0.3, hsv_v=0.5)

New https://pypi.org/project/ultralytics/8.3.78 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
engine/trainer: task=classify, mode=train, model=runs/classify/train112/weights/last.pt, data=classifications/satellite_poolhaving, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train113, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=Non

train: Scanning /home/jupyter/classifications/satellite_poolhaving/train... 330 images, 0 corrupt: 100%|██████████| 330/
val: Scanning /home/jupyter/classifications/satellite_poolhaving/val... 88 images, 0 corrupt: 100%|██████████| 88/88 [00


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 38 weight(decay=0.0), 39 weight(decay=0.0005), 39 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs/classify/train113
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100      2.15G    0.06077         10        224: 100%|██████████| 21/21 [00:00<00:00, 22.01it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00, 94.74it/s]

                   all      0.909          1


...

                   all      0.909          1



100 epochs completed in 0.031 hours.
Optimizer stripped from runs/classify/train113/weights/last.pt, 31.7MB
Optimizer stripped from runs/classify/train113/weights/best.pt, 31.7MB

Validating runs/classify/train113/weights/best.pt...
Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,765,218 parameters, 0 gradients, 41.6 GFLOPs
train: /home/jupyter/classifications/satellite_poolhaving/train... found 330 images in 2 classes ✅ 
val: /home/jupyter/classifications/satellite_poolhaving/val... found 88 images in 2 classes ✅ 
test: /home/jupyter/classifications/satellite_poolhaving/test... found 2742 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00, 90.63it/s]


                   all      0.932          1
Speed: 0.0ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train113


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d73ac8da470>
curves: []
curves_results: []
fitness: 0.9659090936183929
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9318181872367859, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9659090936183929}
save_dir: PosixPath('runs/classify/train113')
speed: {'preprocess': 0.04405866969715465, 'inference': 0.28644908558238635, 'loss': 0.0003738836808638139, 'postprocess': 0.0004253604195334695}
task: 'classify'
top1: 0.9318181872367859
top5: 1.0

In [385]:
modelPath = "runs/classify/train113/weights/best.pt"
poolModel = YOLO(modelPath)

In [386]:
from IPython.display import clear_output

poolTestResults = poolModel.predict("classifications/satellite_poolhaving/test/pool")
nopoolTestResults = poolModel.predict("classifications/satellite_poolhaving/test/no_pool")

clear_output(wait=True)
print(f"done! analyzed {len(poolTestResults)} pool images and {len(nopoolTestResults)} non pool images.")


done! analyzed 12 pool images and 2730 non pool images.


In [387]:
analyzeTestData(poolTestResults, nopoolTestResults, 'pool', 'no_pool')

Accuracy: 2599/2742 = 0.948
Precision: 0.066
Recall: 0.833
F1 Score: 0.123
---
True Positives: 10, False Positives: 141, True Negatives: 2589, False Negatives: 2


In [224]:
def analyzeTestData(positiveTestResults, negativeTestResults, positiveCategory, negativeCategory):
    truePositives = 0
    falsePositives = 0
    trueNegatives = 0
    falseNegatives = 0
    totalTests = 0
    
    for result in positiveTestResults:
        if(result.names[result.probs.top1] == positiveCategory):
            truePositives += 1
        else:
            falseNegatives += 1
        totalTests += 1
    
    for result in negativeTestResults:
        if(result.names[result.probs.top1] == negativeCategory):
            trueNegatives += 1
        else:
            falsePositives += 1
        totalTests += 1

    correctTests = truePositives + trueNegatives
    accuracy = correctTests/totalTests

    precision = truePositives/(truePositives + falsePositives)

    recall = truePositives/(truePositives + falseNegatives)

    f1score = 2 * (precision * recall)/(precision + recall)

    print(f"Accuracy: {correctTests}/{totalTests} = {round(accuracy, 3)}")
    print(f"Precision: {round(precision, 3)}")
    print(f"Recall: {round(recall, 3)}")
    print(f"F1 Score: {round(f1score, 3)}")

    print(f"---\nTrue Positives: {truePositives}, False Positives: {falsePositives}, True Negatives: {trueNegatives}, False Negatives: {falseNegatives}")


# City classification model

In [ ]:
#cityModel = YOLO("yolov8m-cls.pt")
cityModel.train(data="classifications/chi_vs_ind", epochs = 5, device=[0, 1])

In [115]:
'''
cityModel = YOLO('runs/classify/train62/weights/best.pt')

paths =  [streetviewImageFilePathHeader + path + ".png" for path in chicagoDataFrame.loc[60:80, 'StreetviewImageName']]
paths += [streetviewImageFilePathHeader + path + ".png" for path in indianapolisDataFrame.loc[60:80, 'StreetviewImageName']]

results = cityModel.predict(paths)

for i in range(len(paths)):
    result = results[i]
    print("for image ", paths[i], " model predicted", result.names[result.probs.top1], " with confidence ", round(result.probs.top1conf.item(), 3))
'''

cityModel = YOLO('runs/classify/train16/weights/best.pt')

results = cityModel.predict('classifications/chi_vs_ind/test/chicago')
results += cityModel.predict('classifications/chi_vs_ind/test/indianapolis')



image 1/46 /home/jupyter/classifications/chi_vs_ind/test/chicago/v10009_chicago.png: 224x224 chicago 1.00, indianapolis 0.00, .ipynb_checkpoints 0.00, 2.6ms
image 2/46 /home/jupyter/classifications/chi_vs_ind/test/chicago/v10020_chicago.png: 224x224 chicago 1.00, indianapolis 0.00, .ipynb_checkpoints 0.00, 2.6ms
image 3/46 /home/jupyter/classifications/chi_vs_ind/test/chicago/v10035_chicago.png: 224x224 chicago 1.00, indianapolis 0.00, .ipynb_checkpoints 0.00, 2.5ms
image 4/46 /home/jupyter/classifications/chi_vs_ind/test/chicago/v10069_chicago.png: 224x224 chicago 1.00, indianapolis 0.00, .ipynb_checkpoints 0.00, 2.5ms
image 5/46 /home/jupyter/classifications/chi_vs_ind/test/chicago/v10107_chicago.png: 224x224 chicago 1.00, indianapolis 0.00, .ipynb_checkpoints 0.00, 2.5ms
image 6/46 /home/jupyter/classifications/chi_vs_ind/test/chicago/v10110_chicago.png: 224x224 chicago 1.00, indianapolis 0.00, .ipynb_checkpoints 0.00, 2.5ms
image 7/46 /home/jupyter/classifications/chi_vs_ind/test/

# Untrained YOLO v8 detection

In [ ]:
detectionModel = YOLO("yolov8m.pt")

paths = [streetviewImageFilePathHeader + path + ".png" for path in indianapolisDataFrame.loc[:100, 'StreetviewImageName']]
results = model.predict(paths)
for result in results:
    if len(result.boxes) > 0:
        print("---")
        print("Found in ", result.path)
        for box in result.boxes:
            print("---")
            cords = box.xyxy[0].tolist()
            cords = [round(x, 1) for x in cords] # rounds each coordinate
            class_id = result.names[box.cls[0].item()] # gets the name of identified object from dictionary of all objects YOLO can identify
            conf = round(box.conf[0].item(), 2) # rounds to 2 digits
            
            print("Object type:", class_id)
            print("Coordinates:", cords)
            print("Probability:", conf)
            
        result.save("object_detection_results/" + result.path)
            

# Detection Model 
(old, data set too small and annoying to create)

In [ ]:
#training detection model
model = YOLO("yolov8m.pt")
model.train(data=r"./detection_datasets/data.yaml", epochs = 200)

In [ ]:
abandonnessModel = YOLO("runs/detect/train15/weights/best.pt")

results = abandonnessModel.predict("streetview_images/v10070.png")
results[0]

In [ ]:
paths = [streetviewImageFilePathHeader + path + ".png" for path in chicagoDataFrame.loc[100:130, 'StreetviewImageName']]
results = abandonnessModel.predict(paths)

In [ ]:

for i in range(len(results)):
    result = results[i]
    for box in result.boxes:
        print("---")
        cords = box.xyxy[0].tolist()
        cords = [round(x, 1) for x in cords] # rounds each coordinate
        class_id = result.names[box.cls[0].item()] # gets the name of identified object from dictionary of all objects YOLO can identify
        conf = round(box.conf[0].item(), 2) # rounds to 2 digits
        print("Said ID: ",(100 + i), " was ", class_id, ". Actual status: ", chicagoDataFrame.at[100 + i, 'Status'])
        

In [ ]:
#extra training model
abandonnessModel.train(data="datasets/data.yaml", epochs = 5)